# 分かち書き

In [1]:
import spacy
import MeCab

## 前処理

In [2]:
mecab = MeCab.Tagger("")
mecab.parse("")#文字列がGCされるのを防ぐ
text = "依存構造解析の実験を行っています。"

In [3]:
node = mecab.parseToNode(text)
while node:
    word = node.surface
    info = node.feature.split(',')
    if not info[0] in ['BOS/EOS', '記号']:
        print (word, info)
    node = node.next

依存 ['名詞', 'サ変接続', '*', '*', '*', '*', '依存', 'イゾン', 'イゾン']
構造 ['名詞', '一般', '*', '*', '*', '*', '構造', 'コウゾウ', 'コーゾー']
解析 ['名詞', 'サ変接続', '*', '*', '*', '*', '解析', 'カイセキ', 'カイセキ']
の ['助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ']
実験 ['名詞', 'サ変接続', '*', '*', '*', '*', '実験', 'ジッケン', 'ジッケン']
を ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
行っ ['動詞', '自立', '*', '*', '五段・ワ行促音便', '連用タ接続', '行う', 'オコナッ', 'オコナッ']
て ['助詞', '接続助詞', '*', '*', '*', '*', 'て', 'テ', 'テ']
い ['動詞', '非自立', '*', '*', '一段', '連用形', 'いる', 'イ', 'イ']
ます ['助動詞', '*', '*', '*', '特殊・マス', '基本形', 'ます', 'マス', 'マス']


In [4]:
nlp = spacy.load('ja_ginza_nopn')
doc = nlp(text)
print (doc.vector.shape)
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.dep_, token.head.i, token._.pos_detail)

(100,)
0 依存 依存 NOUN compound 2 名詞,普通名詞,サ変可能,*
1 構造 構造 NOUN compound 2 名詞,普通名詞,一般,*
2 解析 解析 NOUN nmod 4 名詞,普通名詞,サ変可能,*
3 の の ADP case 2 助詞,格助詞,*,*
4 実験 実験 NOUN obj 6 名詞,普通名詞,サ変可能,*
5 を を ADP case 4 助詞,格助詞,*,*
6 行っ 行う VERB root 6 動詞,一般,*,*
7 て て SCONJ mark 6 助詞,接続助詞,*,*
8 い 居る AUX aux 6 動詞,非自立可能,*,*
9 ます ます AUX aux 6 助動詞,*,*,*
10 。 。 PUNCT punct 6 補助記号,句点,*,*


In [30]:
class BasePreprocess(object):
    def __init__(self, dict_option='', delete_POS_list=['BOS/EOS', '記号'], extract_POS_list=[], delete_human_name=True):
        """textの前処理(親クラス).

        # Arguments
            dict_option: それぞれのAPIの初期設定に必要な辞書やモデル
            delete_POS_list: 取り除く品詞のリスト
            extract_POS_list: 抽出する品詞のリスト(なければすべて)

        # How to use
            preprocess = Base_preprocess()
            output = preprocess.wakati(text)

        """
        self.api = self._build_api(dict_option)
        self.delete_POS_list = delete_POS_list
        self.extract_POS_list = extract_POS_list
        self.delete_human_name = delete_human_name
        
    def _build_api(self, dict_option):
        """
        Subclasses should override for any actions to run.
        """
        api = None
        return api
    
    def _delete_judge(self, info):
        # 削除に指定されたPOSの単語の場合 True
        if info[0] in self.delete_POS_list:
            return True
        # 人名の場合 True
        if self.delete_human_name and info[2] == '人名':
            return True
        # 抽出に指定されたPOSの単語の場合
        if len(self.extract_POS_list) != 0:
            if info[0] in self.extract_POS_list:
                return False
            else:
                return True
        return False

    def _extract_word(self, POS):
        if len(self.extract_POS_list) == 0 or POS in self.extract_POS_list:
            return True
        else:
            return False
    
    def wakati(self, text):
        """
        Subclasses should override for any actions to run.
        """
        output = None
        return output
    
    def lemma_wakati(self, text):
        """
        Subclasses should override for any actions to run.
        """
        
        output = None
        return output
    

class MecabPreprocess(BasePreprocess):
    def __init__(self, dict_option='', delete_POS_list=['BOS/EOS', '記号'], extract_POS_list=[], delete_human_name=True):
        super(Mecab_preprocess, self).__init__(dict_option, delete_POS_list, extract_POS_list, delete_human_name)
        
    def _build_api(self, dict_option):
        api = MeCab.Tagger(dict_option)
        api.parse('')
        return api
            
    def wakati(self, text):
        node = self.api.parseToNode(text)
        output = ""
        while node:
            word = node.surface
            info = node.feature.split(',')
            if not self._delete_judge(info) and word != '':
                output += word
                output += " "
            node = node.next 
        output = output[:-1]
        return output
    
    def lemma_wakati(self, text):
        node = self.api.parseToNode(text)
        output = ""
        while node:
            info = node.feature.split(',')
            word = info[6]
            if not self._delete_judge(info) and word != '':
                output += word
                output += " "
            node = node.next
        output = output[:-1]
        return output


class GinzaPreprocess(BasePreprocess):
    def __init__(self, dict_option='ja_ginza_nopn', delete_POS_list=['BOS/EOS', '補助記号'], extract_POS_list=[]):
        super(GiNZA_preprocess, self).__init__(dict_option, delete_POS_list, extract_POS_list)
        
    def _build_api(self, dict_option):
        api = spacy.load('ja_ginza_nopn')
        return api
    
    def wakati(self, text):
        doc = self.api(text)
        output = ""
        for sent in doc.sents:
            for token in sent:
                word = token.orth_
                info = token._.pos_detail.split(',')
                if not self._delete_judge(info) and word != '':
                    output += word
                    output += " "
        output = output[:-1]
        return output
    
    def lemma_wakati(self, text):
        doc = self.api(text)
        output = ""
        for sent in doc.sents:
            for token in sent:
                word = token.lemma_
                info = token._.pos_detail.split(',')
                if not self._delete_judge(info) and word != '':
                    output += word
                    output += " "
        output = output[:-1]
        return output

In [43]:
text ='終わらせろaaaaaaaa by facebook'

In [44]:
text.rfind('by')

6

In [50]:
text[:text.rfind('by', 0, 8)]

'終わらせろ '

In [40]:
text = '依存構造解析の実験を行っています。(夏目漱石)'
text[:text.rfind('by', -1, -8)]

'依存構造解析の実験を行っています。(夏目漱石'

In [33]:
preprocess = Mecab_preprocess()
text ='終わらせろ by facebook'
output = preprocess.wakati(text)
output

'立てよ 国民'

In [34]:
preprocess = GiNZA_preprocess()
text ='依存構造解析の実験を行っています。(夏目漱石)'
output = preprocess.wakati(text)
output

'依存 構造 解析 の 実験 を 行っ て い ます'

In [1]:
import sys
sys.path.append('../src/utils/')
import preprocess_function as preprocess_func

In [2]:
preprocess = preprocess_func.MecabPreprocess()
text ='終わらせろ'
output = preprocess.wakati(text)
output

'終わら せろ '

## 実際にCSVに書き込み

In [8]:
base_csv_path = '../data/text_data/crawler/positive_bot3.csv'

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv(base_csv_path)

In [11]:
a = {'re_text': df['text']}
b = pd.DataFrame(a)
b.head()

,re_text
0,ハードルは高ければ高いほどくぐりやすい。
1,「明日は、明日こそは」と、 人はそれをなだめる。この「明日」が、彼を墓場に送り込むその日まで。
2,乗りかけた船には、ためらわず乗ってしまえ。
3,99%ダメ?? あと1%あるやん。
4,人より早く起き、人より努力して、初めて成功の味を真に噛みしめる事ができる。


In [12]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

text_list = df['text']
preprocess = GiNZA_preprocess()
wakati_text_list = []
for text in tqdm(text_list):
    wakati_text_list.append(preprocess.wakati(text))
wakati_df = pd.DataFrame({'wakati_text': wakati_text_list})
wakati_df.head()

,wakati_text
0,ハードル は 高けれ ば 高い ほど くぐり やすい
1,明日 は 明日 こそ は と 人 は それ を なだめる この 明日 が 彼 を 墓場 に ...
2,乗り かけ た 船 に は ためらわ ず 乗っ て しまえ
3,99 % ダメ あと 1 % ある や ん
4,人 より 早く 起き 人 より 努力 し て 初めて 成功 の 味 を 真 に 噛みしめる ...


In [13]:
text_list = df['text']
preprocess = Mecab_preprocess()
mecab_wakati_text_list = []
for text in tqdm(text_list):
    mecab_wakati_text_list.append(preprocess.wakati(text))
mecab_wakati_df = pd.DataFrame({'wakati_text': mecab_wakati_text_list})
mecab_wakati_df.head()

,wakati_text
0,ハードル は 高けれ ば 高い ほど くぐり やすい
1,明日 は 明日 こそ は と 人 は それ を なだめる この 明日 が 彼 を 墓場 に ...
2,乗りかけ た 船 に は ためらわ ず 乗っ て しまえ
3,99% ダメ あと 1% ある やん
4,人 より 早く 起き 人 より 努力 し て 初めて 成功 の 味 を 真に 噛みしめる 事...


In [14]:
pd.concat([df, mecab_wakati_df], axis=1).head()

,created_at,id,screen_name,text,wakati_text
0,Thu Apr 11 13:18:49 +0000 2019,1116329758181085184,positive_bot3,ハードルは高ければ高いほどくぐりやすい。,ハードル は 高けれ ば 高い ほど くぐり やすい
1,Thu Apr 11 11:18:49 +0000 2019,1116299557946585089,positive_bot3,「明日は、明日こそは」と、 人はそれをなだめる。この「明日」が、彼を墓場に送り込むその日まで。,明日 は 明日 こそ は と 人 は それ を なだめる この 明日 が 彼 を 墓場 に ...
2,Thu Apr 11 07:18:48 +0000 2019,1116239156861001728,positive_bot3,乗りかけた船には、ためらわず乗ってしまえ。,乗りかけ た 船 に は ためらわ ず 乗っ て しまえ
3,Thu Apr 11 05:18:47 +0000 2019,1116208955355611136,positive_bot3,99%ダメ?? あと1%あるやん。,99% ダメ あと 1% ある やん
4,Thu Apr 11 03:18:48 +0000 2019,1116178759336337408,positive_bot3,人より早く起き、人より努力して、初めて成功の味を真に噛みしめる事ができる。,人 より 早く 起き 人 より 努力 し て 初めて 成功 の 味 を 真に 噛みしめる 事...


In [15]:
wakati_df = pd.concat([df, mecab_wakati_df], axis=1)

In [58]:
df = pd.read_csv('../data/text_data/crawler/negative/nega_bot.csv')
df.head()

,created_at,id,screen_name,text,wakati_text
0,Mon Apr 15 05:35:35 +0000 2019,1117662734529646592,nega_bot,優しさがつらすぎる,優しさ が つら すぎる
1,Mon Apr 15 05:05:39 +0000 2019,1117655201505169408,nega_bot,終わってる,終わっ てる
2,Mon Apr 15 04:35:35 +0000 2019,1117647635978960896,nega_bot,生きてる意義がわからない,生き てる 意義 が わから ない
3,Mon Apr 15 03:35:37 +0000 2019,1117632542725689345,nega_bot,もう何年も何もしてない,もう 何 年 も 何 も し て ない
4,Mon Apr 15 03:05:37 +0000 2019,1117624993590833152,nega_bot,調査マジしんどい,調査 マジ しんどい
